In [1]:
!pip install datasets
!pip install git+https://github.com/huggingface/transformers accelerate
!pip install --upgrade qwen-vl-utils
!pip install tqdm
!pip install wheel
!pip install flash-attn --no-build-isolation
!pip install torch
!pip install torchvision




[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-lygvltu7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-lygvltu7
  Resolved https://github.com/huggingface/transformers to commit 08e3217bafddc5d11ce0e7369bcfaaabe5501ba5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[no

In [6]:
import pandas as pd

ds=pd.read_csv("clean_qnsdata.csv")

Note that these functions are what was given by huggingface as this is how the model processes videos. Different models have different ways of processing videos.

In [3]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch


# default: Load the model on the available device(s)
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
# )

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
qwenmodel = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "FanqingM/MM-Eureka-Qwen-7B",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)

# # default processer
#qwenprocessor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
min_pixels = 256*28*28
max_pixels = 1280*28*28
qwenprocessor = AutoProcessor.from_pretrained("FanqingM/MM-Eureka-Qwen-7B", min_pixels=min_pixels, max_pixels=max_pixels)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [8]:
def video(prompt, video_path):
  messages = [
      {
          "role": "user",
          "content": [
              {
                  "type": "video",
                  "video": video_path,
                  "max_pixels": 360 * 420,
                  "fps": 1.0,
              },
              {"type": "text", "text": "Answer each question in the list provided using the video. The question list: " + prompt + "Output your answer as a string, with each answer seperated by commas. Your answer to each question:"},
          ],
      }
  ]

  #In Qwen 2.5 VL, frame rate information is also input into the model to align with absolute time.
  # Preparation for inference
  text = qwenprocessor.apply_chat_template(
      messages, tokenize=False, add_generation_prompt=True
  )
  image_inputs, video_inputs, video_kwargs = process_vision_info(messages, return_video_kwargs=True)
  inputs = qwenprocessor(
      text=[text],
      images=image_inputs,
      videos=video_inputs,
      # fps=fps,
      padding=True,
      return_tensors="pt",
      **video_kwargs

  )
  inputs = inputs.to("cuda")

  # Inference
  generated_ids = qwenmodel.generate(**inputs, max_new_tokens=500)
  generated_ids_trimmed = [
      out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
  ]
  output_text = qwenprocessor.batch_decode(
      generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
  )
  return output_text

# def video(questions, video_path):
#     messages_batch = []

#     for prompt in questions:
#         messages = [
#             {
#                 "role": "user",
#                 "content": [
#                     {
#                         "type": "video",
#                         "video": video_path,
#                         "max_pixels": 360 * 420,
#                         "fps": 1.0,
#                     },
#                     {"type": "text", "text": prompt},
#                 ],
#             }
#         ]
#         messages_batch.append(messages)

#     # Prepare text prompts for each question
#     texts = [
#         qwenprocessor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True)
#         for msg in messages_batch
#     ]

#     # Use the first video (same for all) to extract video features
#     image_inputs, video_inputs, video_kwargs = process_vision_info(
#         messages_batch[0], return_video_kwargs=True
#     )

#     # Replicate vision inputs for all questions in the batch
#     image_inputs_batch = [image_inputs[0]] * len(texts)
#     video_inputs_batch = [video_inputs[0]] * len(texts)

#     # Tokenize
#     inputs = qwenprocessor(
#         text=texts,
#         images=image_inputs_batch,
#         videos=video_inputs_batch,
#         padding=True,
#         return_tensors="pt",
#         **video_kwargs
#     ).to("cuda")

#     # Generate
#     generated_ids = qwenmodel.generate(**inputs, max_new_tokens=128)
#     generated_ids_trimmed = [
#         out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
#     ]

#     # Decode
#     output_texts = qwenprocessor.batch_decode(
#         generated_ids_trimmed,
#         skip_special_tokens=True,
#         clean_up_tokenization_spaces=False
#     )

#     return output_texts

# def video(questions, video_path):
#     messages_batch = []

#     for prompt in questions:
#         messages = [
#             {
#                 "role": "user",
#                 "content": [
#                     {
#                         "type": "video",
#                         "video": video_path,
#                         "max_pixels": 360 * 420,
#                         "fps": 1.0,
#                     },
#                     {"type": "text", "text": prompt},
#                 ],
#             }
#         ]
#         messages_batch.append(messages)

#     # Prepare prompt texts
#     texts = [
#         qwenprocessor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True)
#         for msg in messages_batch
#     ]

#     # Use first video prompt for visual input
#     _, video_inputs, video_kwargs = process_vision_info(messages_batch[0], return_video_kwargs=True)

#     # Replicate video input N times
#     video_inputs_batch = [video_inputs[0]] * len(texts)

#     # No image inputs needed in this case
#     image_inputs_batch = [None] * len(texts)

#     # Tokenize
#     inputs = qwenprocessor(
#         text=texts,
#         images=image_inputs_batch,
#         videos=video_inputs_batch,
#         padding=True,
#         return_tensors="pt",
#         **video_kwargs
#     ).to("cuda")

#     # Generate
#     generated_ids = qwenmodel.generate(**inputs, max_new_tokens=128)
#     generated_ids_trimmed = [
#         out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
#     ]

#     # Decode
#     output_texts = qwenprocessor.batch_decode(
#         generated_ids_trimmed,
#         skip_special_tokens=True,
#         clean_up_tokenization_spaces=False
#     )

#     return output_texts

Remember insert videos into your own drive first

In [9]:
# #WARNING: This will delete previous progress!
# import os

# clear_checkpoint = True
# checkpoint_path = "qwenpredictions.json"

# if clear_checkpoint and os.path.exists(checkpoint_path):
#     os.remove(checkpoint_path)
#     print("Checkpoint cleared.")

Checkpoint cleared.


In [7]:
# test ALL data
import os
import json
from tqdm import tqdm
import traceback

checkpoint_path = "qwenanswers.json"



# Load previous checkpoint if exists
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, "r") as f:
        predictions = json.load(f)
    processed_qids = {p['qid'] for p in predictions}
    print(f"Loaded {len(predictions)} predictions from checkpoint.")
else:
    predictions = []
    processed_qids = set()

with torch.no_grad():
    for i, sample in enumerate(tqdm(ds, desc="Processing videos")):
        if sample["qid"] in processed_qids:
            continue  # Skip already processed

        video_path = f"Benchmark-AllVideos-HQ-Encoded-challenge/{sample['video_id']}.mp4"
        question = sample["breakdown"]

        try:
          
            result=video(question,video_path)
            
            torch.cuda.empty_cache()        # Clears unused memory
            torch.cuda.ipc_collect()

        except Exception as e:
            print(f"Error on {sample['video_id']}: {e}")
            predictions.append({
                "qid": sample["qid"],
                "prediction": "Error"
            })
            with open(checkpoint_path, "w") as f:
                json.dump(predictions, f, indent=2)
            traceback.print_exc()
            continue

        print(f"Prediction {len(predictions)}: {result}")
        predictions.append({
            "qid": sample["qid"],
            "prediction": result
        })

        # Save every 5 predictions
        if len(predictions) % 5 == 0:
            with open(checkpoint_path, "w") as f:
                json.dump(predictions, f, indent=2)


with open(checkpoint_path, "w") as f:
    json.dump(predictions, f, indent=2)


with open("qwenanswers.json", "w") as f:
    json.dump(predictions, f, indent=2)






Processing videos:   0%|          | 1/1500 [00:10<4:31:52, 10.88s/it]

Prediction 0: ['The first two people in the video are shown in different settings and performing different actions:\n\n1. The first person is outdoors, sitting at a table with a bottle of beer. The text "You" is superimposed on the image, suggesting that this person is representing the viewer or an unspecified individual.\n\n2. The second person is indoors, sitting at a table with a Coca-Cola bottle. This person is using a knife to puncture the cap of the bottle, which is a common method to open a bottle without a bottle opener.\n\nThe last person in the video is not shown, so there is no action to compare. However, based on the actions of the first two people, the difference could be interpreted as follows:\n\n- The first person is simply holding a bottle and appears to be in a casual setting.\n- The second person is actively opening a bottle using a knife, which is a more deliberate and potentially risky action compared to the first person\'s passive holding of the bottle.\n\nSince t

Processing videos:   0%|          | 2/1500 [00:20<4:12:11, 10.10s/it]

Prediction 1: ["The video shows three different individuals performing actions that are somewhat unusual or unexpected:\n\n1. The first person is holding a bottle of beer and appears to be in an outdoor setting, possibly at a bus stop. The action here is relatively normal and everyday.\n\n2. The second person is indoors, sitting at a table with a Coca-Cola bottle. This person is using a knife to open the bottle, which is an unconventional method for opening a bottle. Typically, one would use a bottle opener or simply twist the cap off.\n\n3. The third person is also indoors, and their action involves using a knife to open a Coca-Cola bottle in a similar manner to the second person. This action is again unusual and not the standard way to open a bottle.\n\nThe actions of the last two individuals (the second and third) are similar in that they both use a knife to open a Coca-Cola bottle, which is an atypical method. The first person's action is more typical and does not involve any unusu

Processing videos:   0%|          | 3/1500 [00:27<3:35:08,  8.62s/it]

...

Prediction 17: ["The video shows a machine that is cutting zucchini into thin slices. The blades of the machine are indeed spinning at a high speed, which is a common feature in food processing machines designed for slicing. The stroboscopic effect, where objects appear to be moving in a continuous loop due to the rapid movement and the camera's frame rate, can create an illusion of the slices being cut in a continuous motion. However, the actual cutting process is not a stroboscopic effect but rather the result of the high-speed spinning of the blades, which割 (cut) the zucchini into thin slices.\n\nFinal answer: Yes, the reason the machine cuts the zucchini so thinly is because the blades are spinning at a high speed, which is a common feature in food processing machines designed for slicing. The stroboscopic effect is a visual phenomenon that can occur when the speed of the blades and the camera's frame rate align in a way that creates the illusion of continuous motion, but it is not

Processing videos:   1%|▏         | 19/1500 [02:23<3:59:22,  9.70s/it]

Prediction 18: ['The machine in the video appears to be a vegetable slicer, which is designed to cut vegetables into thin slices using a rotating blade. The video shows that the zucchini is being sliced into thin rounds, which suggests that the machine is capable of producing thin cuts. The sharpness of the blade is likely a key factor in achieving these thin slices, as a dull blade would not produce the same level of thinness. Additionally, the speed of the machine can affect the thickness of the slices; a slower speed might allow for more control and potentially thinner slices, but the primary factor in this case seems to be the sharpness of the blade.\n\nFinal answer: Yes, the machine is able to cut the zucchini thinly because of the sharpness of the blade, even at a slower speed. The video demonstrates that the zucchini is being sliced into thin rounds, which indicates that the blade is sharp enough to produce thin cuts. The speed of the machine may also play a role, but the sharpn

Processing videos:   1%|▏         | 20/1500 [02:28<3:27:38,  8.42s/it]

Prediction 19: ['A']


In [8]:
import json
import csv

# Load the original predictions
with open("qwenanswers.json", "r") as f:
    predictions = json.load(f)

# Prepare rows (preserve qid, clean prediction text)
rows = []
for item in predictions:
    qid = item["qid"].strip()
    pred = str(item["prediction"]).strip()
    rows.append([qid, pred])

# Save directly to CSV
with open("qwenanswers.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["qid", "pred"])
    writer.writerows(rows)